In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier 

In [2]:
df = pd.read_csv("dump2.csv")
df.head(5)

,id,box_id,device_type,false_character,hold_for,key_pressed,location,long_pressed_equivalent,name,platform,pressed_after,timestamp,user_id
0,902,#text-area-1,Computer/Laptop,f,754,Shift,16,2,Kedar,MacIntel,0,1584257309690,ef70b84c-be3d-434b-9cbe-55a19ce557a0
1,903,#text-area-1,Computer/Laptop,f,63,h,72,1,Kedar,MacIntel,457,1584257310901,ef70b84c-be3d-434b-9cbe-55a19ce557a0
2,904,#text-area-1,Computer/Laptop,f,80,i,73,1,Kedar,MacIntel,105,1584257311070,ef70b84c-be3d-434b-9cbe-55a19ce557a0
3,905,#text-area-1,Computer/Laptop,f,86,s,83,1,Kedar,MacIntel,19,1584257311169,ef70b84c-be3d-434b-9cbe-55a19ce557a0
4,906,#text-area-1,Computer/Laptop,f,72,,32,1,Kedar,MacIntel,54,1584257311309,ef70b84c-be3d-434b-9cbe-55a19ce557a0


In [3]:
df["name"].value_counts()

Kedar             2542
Vikas Ramaneti     945
Sope Ogundipe      810
Kunal Sonar        769
Name: name, dtype: int64

In [4]:
df[df['name'] == 'Kedar']['user_id'].value_counts()

ef70b84c-be3d-434b-9cbe-55a19ce557a0    863
7ce1efdf-a289-4a3c-9aa9-a3ff66cf6433    844
c8dbf287-6ae4-4fba-bba9-7900a6637d48    835
Name: user_id, dtype: int64

In [14]:
data = df[~df['user_id'].isin(['ef70b84c-be3d-434b-9cbe-55a19ce557a0','7ce1efdf-a289-4a3c-9aa9-a3ff66cf6433'])]
data.shape

(3359, 13)

In [15]:
data["name"].value_counts()

Vikas Ramaneti    945
Kedar             835
Sope Ogundipe     810
Kunal Sonar       769
Name: name, dtype: int64

In [16]:
data[['name','false_character','long_pressed_equivalent','id']].groupby(['name','false_character','long_pressed_equivalent']).count()

id
name           false_character long_pressed_equivalent     
Kedar          f               1                        714
                               2                         79
               t               1                         38
                               2                          4
Kunal Sonar    f               1                        613
                               2                        125
                               3                          1
               t               1                         30
Sope Ogundipe  f               1                        702
                               2                         71
                               8                          1
               t               1                         30
                               2                          4
                               5                          1
                               6                          1
Vikas Ramaneti f               1                        726
                               2                         51
               t               1                        164
                               2                          4

In [17]:
labelEncoder = preprocessing.LabelEncoder()
data['false_character'] = labelEncoder.fit_transform(data['false_character'])
false_character_map = {labelEncoder.classes_[i] : i for i in range(len(labelEncoder.classes_))}
data['name'] = labelEncoder.fit_transform(data['name'])
name_map = {labelEncoder.classes_[i] : i for i in range(len(labelEncoder.classes_))}
data['box_id'] = labelEncoder.fit_transform(data['box_id'])
box_map = {labelEncoder.classes_[i] : i for i in range(len(labelEncoder.classes_))}
data.head(5)

C:\Users\sopeo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\sopeo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\sopeo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

,id,box_id,device_type,false_character,hold_for,key_pressed,location,long_pressed_equivalent,name,platform,pressed_after,timestamp,user_id
863,1802,0,Computer/Laptop,0,283,T,84,2,2,Win32,0,1584560377463,3aed9f41-ae01-46da-bcbf-b92b76caf99a
864,1803,0,Computer/Laptop,0,70,h,72,1,2,Win32,111,1584560377857,3aed9f41-ae01-46da-bcbf-b92b76caf99a
865,1804,0,Computer/Laptop,0,45,i,73,1,2,Win32,146,1584560378073,3aed9f41-ae01-46da-bcbf-b92b76caf99a
866,1805,0,Computer/Laptop,0,60,s,83,1,2,Win32,36,1584560378154,3aed9f41-ae01-46da-bcbf-b92b76caf99a
867,1806,0,Computer/Laptop,0,89,,32,1,2,Win32,353,1584560378567,3aed9f41-ae01-46da-bcbf-b92b76caf99a


In [18]:
false_character_map, name_map, box_map

({'f': 0, 't': 1},
 {'Kedar': 0, 'Kunal Sonar': 1, 'Sope Ogundipe': 2, 'Vikas Ramaneti': 3},
 {'#text-area-1': 0,
  '#text-area-2': 1,
  '#text-area-3': 2,
  '#text-area-test': 3})

In [20]:
df_reduced = data[['name','false_character','hold_for','location','long_pressed_equivalent','pressed_after']]
df_reduced.head()

,name,false_character,hold_for,location,long_pressed_equivalent,pressed_after
863,2,0,283,84,2,0
864,2,0,70,72,1,111
865,2,0,45,73,1,146
866,2,0,60,83,1,36
867,2,0,89,32,1,353


In [21]:
df_reduced['name'].value_counts()

3    945
0    835
2    810
1    769
Name: name, dtype: int64

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df_reduced.iloc[:,1:], df_reduced.iloc[:,0], test_size=0.2, stratify=df_reduced.iloc[:,0], random_state=109)

In [23]:
grouped = df_reduced.groupby('name')
balance_df = grouped.apply(lambda x : x.sample(grouped.size().min())).reset_index(drop=True)
balance_df['name'].value_counts()

3    769
1    769
2    769
0    769
Name: name, dtype: int64

In [24]:
balance_df[['name','false_character','long_pressed_equivalent','hold_for']].groupby(['name','false_character','long_pressed_equivalent']).count()

hold_for
name false_character long_pressed_equivalent          
0    0               1                             659
                     2                              71
     1               1                              35
                     2                               4
1    0               1                             613
                     2                             125
                     3                               1
     1               1                              30
2    0               1                             667
                     2                              66
                     8                               1
     1               1                              29
                     2                               4
                     5                               1
                     6                               1
3    0               1                             596
                     2                              39
     1               1                             130
                     2                               4

In [25]:
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(balance_df.iloc[:,1:], balance_df.iloc[:,0], test_size=0.2, stratify=balance_df.iloc[:,0] ,random_state=10)
X_train_bal.shape, X_test_bal.shape, y_train_bal.shape, y_test_bal.shape

((2460, 5), (616, 5), (2460,), (616,))

### SVM Multiclass Classification on Unbalanced Data

In [26]:
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 

In [27]:
# model accuracy for X_test  
accuracy = svm_model_linear.score(X_test, y_test) 
print("Accuracy : ", (accuracy*100).round(3))

Accuracy :  41.518


In [28]:
# creating a confusion matrix 
cm = confusion_matrix(y_test, svm_predictions) 
print('Confusion Matrix :')
print(cm)

Confusion Matrix :
[[ 66  21  14  66]
 [ 43  66   1  44]
 [ 34  15  14  99]
 [ 36  16   4 133]]


In [29]:
print('Classification Report : ')
print(classification_report(y_test, svm_predictions))

Classification Report : 
              precision    recall  f1-score   support

           0       0.37      0.40      0.38       167
           1       0.56      0.43      0.49       154
           2       0.42      0.09      0.14       162
           3       0.39      0.70      0.50       189

    accuracy                           0.42       672
   macro avg       0.44      0.40      0.38       672
weighted avg       0.43      0.42      0.38       672



### SVM Multiclass Classification on Balanced Data

In [30]:
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train_bal, y_train_bal) 
svm_predictions = svm_model_linear.predict(X_test_bal) 

In [31]:
accuracy = svm_model_linear.score(X_test_bal, y_test_bal) 
print("Accuracy : ", (accuracy*100).round(3))

Accuracy :  45.942


In [32]:
cm = confusion_matrix(y_test_bal, svm_predictions) 
print('Confusion Matrix :')
print(cm)

Confusion Matrix :
[[81 29 37  7]
 [39 84 27  4]
 [39 17 92  6]
 [48 16 64 26]]


In [33]:
print('Classification Report : ')
print(classification_report(y_test_bal, svm_predictions))

Classification Report : 
              precision    recall  f1-score   support

           0       0.39      0.53      0.45       154
           1       0.58      0.55      0.56       154
           2       0.42      0.60      0.49       154
           3       0.60      0.17      0.26       154

    accuracy                           0.46       616
   macro avg       0.50      0.46      0.44       616
weighted avg       0.50      0.46      0.44       616



### KNN Multiclass Classification on Unbalanced Data

In [34]:
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train) 

In [35]:
# accuracy on X_test 
accuracy = knn.score(X_test, y_test) 
print("Accuracy : ", (accuracy*100).round(3))

Accuracy :  46.429


In [36]:
# creating a confusion matrix 
knn_predictions = knn.predict(X_test)  
cm = confusion_matrix(y_test, knn_predictions) 

In [37]:
print('Confusion Matrix :')
print(cm)

Confusion Matrix :
[[78 25 26 38]
 [37 83 17 17]
 [37 17 66 42]
 [46 25 33 85]]


In [38]:
print('Classification Report : ')
print(classification_report(y_test, knn_predictions))

Classification Report : 
              precision    recall  f1-score   support

           0       0.39      0.47      0.43       167
           1       0.55      0.54      0.55       154
           2       0.46      0.41      0.43       162
           3       0.47      0.45      0.46       189

    accuracy                           0.46       672
   macro avg       0.47      0.47      0.47       672
weighted avg       0.47      0.46      0.46       672



### KNN Multiclass Classification on Balanced Data

In [39]:
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train_bal, y_train_bal) 

In [40]:
accuracy = knn.score(X_test_bal, y_test_bal) 
print("Accuracy : ", (accuracy*100).round(3))

Accuracy :  42.857


In [41]:
knn_predictions = knn.predict(X_test_bal)  
cm = confusion_matrix(y_test_bal, knn_predictions) 

In [42]:
print('Confusion Matrix :')
print(cm)

Confusion Matrix :
[[74 31 20 29]
 [34 89 17 14]
 [41 26 50 37]
 [47 22 34 51]]


In [43]:
print('Classification Report : ')
print(classification_report(y_test_bal, knn_predictions))

Classification Report : 
              precision    recall  f1-score   support

           0       0.38      0.48      0.42       154
           1       0.53      0.58      0.55       154
           2       0.41      0.32      0.36       154
           3       0.39      0.33      0.36       154

    accuracy                           0.43       616
   macro avg       0.43      0.43      0.42       616
weighted avg       0.43      0.43      0.42       616

